In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

#
# Model pipeline
#
D = 2 ** 10

features = [str(i) for i in range(1,40)]


fields = ['id','label'] + numeric_features + categorical_features + ["day_number"]

# Now we have a full prediction pipeline.
model = Pipeline(steps=[
    ('logisticregression', LogisticRegression())
])

def get_x(csv_row, D):
    x = [0]  # 0 is the index of the bias term
    for key, value in csv_row.items():
        
        index = int(str(value) + key[1:], 16) % D  # weakest hash ever ;)
        x.append(index)
    return x  # x contains indices of features that have a value of 1

In [79]:
import os, sys
import logging

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
from sklearn.metrics import log_loss

In [62]:
train_path='/home/users/datasets/criteo_train1'

In [92]:
X_train = []
Y_train = []
X_test = []
Y_test = []

for t, row in enumerate(open(train_path)):
    if t%10==0:
        if t%3==0:
            row = dict(zip(fields, row.replace("\n",'').split('\t')))
            y = 1. if row['label'] == '1' else 0.
            
            del row['label']  # can't let the model peek the answer
            del row['id']  # we don't need the Id
            del row['day_number']

            # step 1, get the hashed features
            x = get_x(row, D)
            x_row = np.zeros(D)
            x_row[x] = 1
            X_test.append(x_row)
            Y_test.append(y)
        else:
            row = dict(zip(fields, row.replace("\n",'').split('\t')))
            y = 1. if row['label'] == '1' else 0.

            del row['label']  # can't let the model peek the answer
            del row['id']  # we don't need the Id
            del row['day_number']

            # step 1, get the hashed features
            x = get_x(row, D)
            x_row = np.zeros(D)
            x_row[x] = 1
            X_train.append(x_row)
            Y_train.append(y)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [104]:
model.fit(X_train, Y_train)

/opt/conda/envs/dsenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='warn', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [95]:
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None, nrows=100, keep_default_na=False
)

In [96]:
df = pd.read_csv(train_path, **read_opts)

In [97]:
df.drop(columns = ['label'], inplace=True)

In [98]:
df.head()

,id,if1,if2,if3,if4,if5,if6,if7,if8,if9,...,cf18,cf19,cf20,cf21,cf22,cf23,cf24,cf25,cf26,day_number
0,0,,600,1,342,9,0,0,18,1,...,b8170bba,9512c20b,9b6712fa,e611fb63,27fc20dc,,201d7147,57e36578,b757e957,day_0
1,1,,1424,1,1291,7,0,0,0,16,...,a1eb1511,9512c20b,fc33311c,f766202e,e1c2c118,,4ffc2977,ff654802,b757e957,day_0
2,2,3,239,3,,8,0,0,139,0,...,b8170bba,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,10c76e2c,30436bfc,e1be5ef2,day_0
3,3,11,537,3,,6,0,0,366,2,...,a1eb1511,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,86c2a370,30436bfc,e1be5ef2,day_0
4,4,3,124,39,398,3,0,0,1,40,...,a1eb1511,9512c20b,47849e55,73b3f46d,d994ba60,,d3d5ac76,ff654802,b757e957,day_0


In [81]:
pred = []

In [99]:
for index, row in df.iterrows():
    test=row

In [100]:
del row['id']  # we don't need the Id
del row['day_number']

In [101]:
D = 2 ** 10

In [108]:
x = get_x(row, D)
x_row = np.zeros(D)
x_row[x] = 1
pred.append(model.predict(x_row.reshape(1, -1))[0])

In [109]:
pred

[array([0.]), 0.0]

In [110]:
read_opts=dict(
        sep='\t', names=fields, index_col=False, header=None, nrows=100, keep_default_na=False
)
df = pd.read_csv(train_path, **read_opts)
df.drop(columns = ['label'], inplace=True)

pred = []
for index, row in df.iterrows():
    ids = df['id']
    del row['id']  # we don't need the Id
    del row['day_number']
    x = get_x(row, D)
    x_row = np.zeros(D)
    x_row[x] = 1
    pred.append(model.predict(x_row.reshape(1, -1))[0]) 
out = zip(ids, pred)

In [113]:
print("\n".join(["{0},{1}".format(*i) for i in out]))

0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0
11,0.0
12,0.0
13,0.0
14,0.0
15,0.0
16,0.0
17,0.0
18,0.0
19,0.0
20,0.0
21,0.0
22,0.0
23,0.0
24,0.0
25,0.0
26,0.0
27,0.0
28,0.0
29,0.0
30,0.0
31,0.0
32,0.0
33,0.0
34,0.0
35,0.0
36,0.0
37,0.0
38,0.0
39,0.0
40,0.0
41,0.0
42,0.0
43,0.0
44,0.0
45,0.0
46,0.0
47,0.0
48,0.0
49,0.0
50,0.0
51,0.0
52,0.0
53,0.0
54,0.0
55,0.0
56,0.0
57,0.0
58,0.0
59,0.0
60,0.0
61,0.0
62,0.0
63,0.0
64,0.0
65,0.0
66,0.0
67,0.0
68,0.0
69,0.0
70,0.0
71,0.0
72,0.0
73,0.0
74,0.0
75,0.0
76,0.0
77,0.0
78,0.0
79,0.0
80,0.0
81,0.0
82,0.0
83,0.0
84,0.0
85,0.0
86,0.0
87,0.0
88,0.0
89,0.0
90,0.0
91,0.0
92,0.0
93,0.0
94,0.0
95,0.0
96,0.0
97,0.0
98,0.0
99,0.0
